In [1]:
import requests
import csv
from bs4 import BeautifulSoup
import pandas as pd 

In [3]:
link='https://books.toscrape.com/catalogue/category/books_1/index.html'
soup=BeautifulSoup(requests.get(link).text,'html.parser')
categories=[]
for cat in soup.find('ul',attrs={'class':'nav nav-list'}).find_all('a')[1:]:
    #print(cat)
    if ' 'in cat.get_text().strip():
        x=cat.get_text().strip().replace(" ", "-")
        categories.append(x)
    else:  
        categories.append(cat.get_text().strip())
#categories     

In [4]:
links=[]
for i in range (len(categories)):
    link =f'https://books.toscrape.com/catalogue/category/books/{categories[i]}_{i+2}/index.html'
    links.append(link.lower())

In [5]:
ratings={'One':1,'Two':2,'Three':3,'Four':4,'Five':5}
files=[]
for link in links:
    soup=BeautifulSoup(requests.get(link).text,'html.parser')
    file_name=soup.find('div',attrs={'class':'page-header action'}).get_text().strip()+'.csv'
    files.append(file_name)
    with open (file_name,'w',newline='',encoding='utf-8') as f:
        writer=csv.writer(f)
        writer.writerow(['book name','book price','book rating'])
        if soup.find('ul',attrs={'class':'pager'}):
            n=int(soup.find('ul',attrs={'class':'pager'}).find('li').get_text().split('of')[1].strip())
            for i in range (1,n+1):
                link=link[:link.rfind('/')+1]+f'page-{i}.html'
                soup=BeautifulSoup(requests.get(link).text,'html.parser')
                for book in soup.find_all('article',attrs={'class':'product_pod'}): 
                    book_name=book.find_all('a')[1].get('title')
                    book_price=book.find('p',attrs={'class':'price_color'}).get_text()[2:]
                    book_rating=ratings[book.find('p',attrs={'class':'star-rating'}).get('class')[1]]
                    writer.writerow([book_name,book_price,book_rating])

        else:
            for book in soup.find_all('article',attrs={'class':'product_pod'}): 
                    book_name=book.find_all('a')[1].get('title')
                    book_price=book.find('p',attrs={'class':'price_color'}).get_text()[2:]
                    book_rating=ratings[book.find('p',attrs={'class':'star-rating'}).get('class')[1]]
                    writer.writerow([book_name,book_price,book_rating])

            


In [6]:
for file in files:
    print (file)
    df=pd.read_csv(file)
    print(df)
    print('***********************************************************')

Travel.csv
                                            book name  book price  book rating
0                             It's Only the Himalayas       45.17            2
1   Full Moon over Noahâs Ark: An Odyssey to Mou...       49.43            4
2   See America: A Celebration of Our National Par...       48.87            3
3   Vagabonding: An Uncommon Guide to the Art of L...       36.94            2
4                                Under the Tuscan Sun       37.33            3
5                                  A Summer In Europe       44.34            2
6                            The Great Railway Bazaar       30.54            1
7                    A Year in Provence (Provence #1)       56.88            4
8   The Road to Little Dribbling: Adventures of an...       23.21            1
9           Neither Here nor There: Travels in Europe       38.95            3
10                 1,000 Places to See Before You Die       26.08            5
*****************************************

                                            book name  book price  book rating
0   The Most Perfect Thing: Inside (and Outside) a...       42.96            4
1   Immunity: How Elie Metchnikoff Changed the Cou...       57.36            5
2   Sorting the Beef from the Bull: The Science of...       44.74            4
3   Tipping Point for Planet Earth: How Close Are ...       37.55            1
4   The Fabric of the Cosmos: Space, Time, and the...       55.91            1
5   Diary of a Citizen Scientist: Chasing Tiger Be...       28.41            1
6                               The Origin of Species       10.01            4
7                                    The Grand Design       13.76            3
8     Peak: Secrets from the New Science of Expertise       16.28            2
9   The Elegant Universe: Superstrings, Hidden Dim...       13.03            4
10  The Disappearing Spoon: And Other True Tales o...       57.35            5
11  Surely You're Joking, Mr. Feynman!: Adventures..